In [ ]:
import pandas as pd
import numpy as np
import time
import sys
sys.path.append('tools/tensorflow-DeepFM-master/')

import DeepFM

import tensorflow as tf

from sklearn.model_selection import KFold

In [ ]:
trainFile = 'round1_ijcai_18_train_20180301.txt'
testFile = 'round1_ijcai_18_test_a_20180301.txt'
dataDir = '../../Data/advertisment/Raw/'

In [ ]:
dfTrain = pd.read_table(dataDir+trainFile,sep=' ')
dfTest = pd.read_table(dataDir+testFile,sep=' ')

In [ ]:
featInput = []
keyList = ['instance_id','item_id','user_id','context_id','shop_id','is_trade']
for var in dfTrain.columns:
    if not var in keyList and dfTrain[var].dtypes != 'object':
        featInput.append(var)
print(featInput)
print(len(featInput))

In [ ]:
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return value.tm_hour

In [ ]:
dfTrain['context_timestamp_hour'] = dfTrain['context_timestamp'].map(timestamp_datetime)
try: featInput.remove('context_timestamp')
except: print('context_timestamp has been deleted')
if not 'context_timestamp_hour' in featInput:
    featInput.append('context_timestamp_hour')

In [ ]:
dummyInput = []
removeInput = []
for var in featInput:
    if 'int' in str(dfTrain[var].dtypes):
        print(var)
        dfTrain[var].replace({-1:np.nan},inplace=True)
        dfTmp = pd.get_dummies(dfTrain[var],var)
        dfTrain = pd.concat([dfTrain,dfTmp],axis=1)
        try:
            removeInput.append(var)
            dummyInput +=dfTmp.columns.tolist()
        except:
            print('Removed')
for var in removeInput:
    featInput.remove(var)
featInput+=dummyInput

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
for trainIndex,testIndex in cv.split(dfTrain[keyList]):
    trainX,testX = dfTrain.loc[trainIndex,featInput],dfTrain.loc[testIndex,featInput]
    trainY,testY = dfTrain.loc[trainIndex,'is_trade'],dfTrain.loc[testIndex,'is_trade']

In [ ]:
dfm_params = {
    "use_fm": True,
    "use_deep": True,
    "embedding_size": 8,
    "dropout_fm": [1.0, 1.0],
    "deep_layers": [32, 32],
    "dropout_deep": [0.5, 0.5, 0.5],
    "deep_layers_activation": tf.nn.relu,
    "epoch": 30,
    "batch_size": 1024,
    "learning_rate": 0.001,
    "optimizer_type": "adam",
    "batch_norm": 1,
    "batch_norm_decay": 0.995,
    "l2_reg": 0.01,
    "verbose": True,
    "random_seed": 42
}


In [ ]:
dfm_params["feature_size"] = fd.feat_dim
dfm_params["field_size"] = len(Xi_train[0])

In [ ]:
dfm = DeepFM.DeepFM(**dfm_params)